In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error, mean_squared_error

%matplotlib inline

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')

In [4]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [5]:
movies['genres_formatted'] = movies.genres.apply(change_string)

In [6]:
movies=movies.drop(['genres'], axis=1)

In [7]:
tags['tag_new'] = tags[['userId','movieId','tag']].groupby(['userId','movieId'])['tag'].transform(lambda x: ' '.join(x))
tags=tags[['userId','movieId','tag_new']].drop_duplicates()

In [8]:
from scipy import stats

In [9]:
raiting_aggr= ratings.groupby(['movieId'])[['rating']].count()
raiting_aggr.columns=['count_']
raiting_aggr['mean_']= ratings.groupby(['movieId'])[['rating']].mean()
raiting_aggr['median_']= ratings.groupby(['movieId'])[['rating']].median()
raiting_aggr['var_']= ratings.groupby(['movieId'])[['rating']].var()
raiting_aggr['mode']=ratings.groupby(['movieId']).rating.agg(lambda x: stats.mode(x)[0])

In [10]:
raiting_aggr.head()

,count_,mean_,median_,var_,mode
movieId,,,,,
1,215,3.920930,4.0,0.696990,4.0
2,110,3.431818,3.5,0.777419,4.0
3,52,3.259615,3.0,1.112651,3.0
4,7,2.357143,3.0,0.726190,3.0
5,49,3.071429,3.0,0.822917,3.0


In [11]:
x = pd.merge(ratings[['userId','movieId','rating']], raiting_aggr, on='movieId', how='left')

In [12]:
y=x.rating

In [13]:
x=x.drop(['rating'], axis=1)
x=x.drop(['userId'], axis=1)
x=x.drop(['movieId'], axis=1)

In [14]:
x.head()

,count_,mean_,median_,var_,mode
0,215,3.920930,4.0,0.696990,4.0
1,52,3.259615,3.0,1.112651,3.0
2,102,3.946078,4.0,0.667856,4.0
3,203,3.975369,4.0,0.850875,4.0
4,204,4.237745,4.5,0.641475,5.0


In [15]:
x.isnull().any()

count_     False
mean_      False
median_    False
var_        True
mode       False
dtype: bool

In [16]:
x.var_  = x.var_.fillna(0)

In [17]:
x_means=x.copy()

In [19]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=42)

In [20]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  0.8775043469562994
